# Setup

In [2]:
import anndata as ad
import scanpy as sc
import pandas as pd
import fast_matrix_market as fmm
import scdrs
import csv

# Preparations

In [16]:
dat = fmm.mmread("all_cells.mtx")
cellIds = pd.read_csv("all_cells.cells", header = None)
genes = pd.read_csv("all_cells.genes", header = None)
anno = pd.read_csv("all_cells.annotation", header = None)

disease = cellIds[0].str.split("!!").apply(lambda x: x[0]).str.split("_").apply(lambda x: x[0])

In [17]:
adata = ad.AnnData(X = dat.tocsr())
adata.obs_names = cellIds[0]
adata.var_names = genes[0]
adata.obs["cell_type"] = pd.Categorical(anno[1])
adata.obs["disease"] = pd.Categorical(disease)
adata

AnnData object with n_obs × n_vars = 117281 × 25734
    obs: 'cell_type', 'disease'

In [14]:
# Add graph, necessary for downstream
sc.pp.neighbors(adata)

         Falling back to preprocessing with `sc.pp.pca` and default params.


/work/miniconda3/envs/scdrs/lib/python3.12/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/work/miniconda3/envs/scdrs/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# Save
adata.write_h5ad("all_cells.h5ad")

# Downstream analyses, not weighted

In [2]:
adata = ad.read_h5ad("all_cells.h5ad")

Munging was performed according to the scDRS documentation. We provide the munge file.

In [4]:
# Preprocessing
dict_gs = scdrs.util.load_gs(
    "gwas/nalls/munge.gs",
    src_species="human",
    dst_species="human",
    to_intersect=adata.var_names,
)

scdrs.preprocess(adata, n_mean_bin=20, n_var_bin=20, copy=False)

In [19]:
# Compute scores
dict_df_score = dict()
for trait in dict_gs:
    gene_list, gene_weights = dict_gs[trait]
    dict_df_score[trait] = scdrs.score_cell(
        data=adata,
        gene_list=gene_list,
        gene_weight=gene_weights,
        ctrl_match_key="mean_var",
        n_ctrl=1000,
        weight_opt="vs",
        return_ctrl_raw_score=False,
        return_ctrl_norm_score=True,
        verbose=True,
    )

# scdrs.method.score_cell summary:
    n_cell=117191, n_gene=25734,
    n_disease_gene=833,
    n_ctrl=1000, n_genebin=200,
    ctrl_match_key='mean_var',
    weight_opt='vs',
    return_ctrl_raw_score=False,
    return_ctrl_norm_score=True,
    random_seed=0, verbose=True,
    save_intermediate=None,
# scdrs.method.score_cell: use 833 overlapping genes for scoring


Computing control scores: 100%|██████████| 1000/1000 [18:36<00:00,  1.12s/it]


In [21]:
# Downstream analyses
df_stats = dict()
df_stats["cell_type"] = scdrs.method.downstream_group_analysis(
    adata=adata,
    df_full_score=dict_df_score["PD"],
    group_cols=["cell_type"],
)

df_stats["disease"] = scdrs.method.downstream_group_analysis(
    adata=adata,
    df_full_score=dict_df_score["PD"],
    group_cols=["disease"],
)

/work/miniconda3/envs/scdrs/lib/python3.12/site-packages/scdrs/method.py:957: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for group, df_group in df_meta.groupby(groupby):
/work/miniconda3/envs/scdrs/lib/python3.12/site-packages/scdrs/method.py:957: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for group, df_group in df_meta.groupby(groupby):


In [43]:
# This was not used, so not provided
dname = "disease"
with open("gwas/nalls/downstream_disease.tsv", 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=df_stats[dname].keys(), delimiter='\t')
    #writer.writeheader()
    writer.writerow(df_stats[dname])

In [44]:
df_stats[dname]

{'disease':         n_cell  n_ctrl  assoc_mcp  assoc_mcz  hetero_mcp  hetero_mcz  \
 group                                                                  
 CTRL   51796.0  1000.0   0.005994   3.528065    0.002997    3.660576   
 MSA    22678.0  1000.0   0.013986   2.611625    0.000999    4.481502   
 PD     42717.0  1000.0   0.006993   3.170693    0.002997    3.652311   
 
        n_fdr_0.05  n_fdr_0.1  n_fdr_0.2  
 group                                    
 CTRL          0.0       14.0      160.0  
 MSA           0.0        1.0       60.0  
 PD            0.0       19.0      187.0  }

In [45]:
dname = "cell_type"
with open("gwas/nalls/downstream_celltype.tsv", 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=df_stats[dname].keys(), delimiter='\t')
    #writer.writeheader()
    writer.writerow(df_stats[dname])

In [46]:
df_stats[dname]

{'cell_type':                         n_cell  n_ctrl  assoc_mcp  assoc_mcz  hetero_mcp  \
 group                                                                      
 Astrocytes             10766.0  1000.0   0.061938   1.613148    0.088911   
 Exc. neurons             583.0  1000.0   0.016983   2.620348    0.034965   
 Immune                   435.0  1000.0   0.013986   2.510072    0.177822   
 Inh. neurons            2846.0  1000.0   0.093906   1.417235    0.208791   
 MSN                     5018.0  1000.0   0.200799   0.877010    0.261738   
 Microglia               6252.0  1000.0   0.003996   3.080197    0.139860   
 OPCs                    5512.0  1000.0   0.023976   2.226710    0.020979   
 Oligodendrocytes       84419.0  1000.0   0.999001  -2.781535    0.116883   
 PVMs                     677.0  1000.0   0.005994   2.972738    0.002997   
 Pericytes/endothelial    683.0  1000.0   0.002997   3.124566    0.153846   
 
                        hetero_mcz  n_fdr_0.05  n_fdr_0.1  n_

In [7]:
dict_df_score = dict()
dict_df_score["PD"] = pd.read_csv("gwas/nalls/PD.full_score.gz", sep = "\t")
dict_df_score["PD"] = dict_df_score["PD"].set_index(dict_df_score["PD"].iloc[:, 0])
dict_df_score["PD"].drop(dict_df_score["PD"].columns[0], axis = 1, inplace = True)
dict_df_score["PD"]

,raw_score,norm_score,mc_pval,pval,nlog10_pval,zscore,ctrl_norm_score_0,ctrl_norm_score_1,ctrl_norm_score_2,ctrl_norm_score_3,...,ctrl_norm_score_990,ctrl_norm_score_991,ctrl_norm_score_992,ctrl_norm_score_993,ctrl_norm_score_994,ctrl_norm_score_995,ctrl_norm_score_996,ctrl_norm_score_997,ctrl_norm_score_998,ctrl_norm_score_999
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
CTRL_037!!CATGAGTTCCACACAA-1,0.157448,1.406709,0.079920,0.083358,1.079052,1.382832,0.404423,-0.333098,0.357902,1.870497,...,-0.491384,1.319264,0.286928,-0.508785,-0.140749,-0.053322,0.384879,-0.049660,-1.808275,0.544533
CTRL_039!!ACGGTTACAACACACT-1,0.158600,2.936088,0.002997,0.002939,2.531734,2.754461,0.582191,-0.396215,-1.573622,1.361310,...,-1.571017,-1.622350,-0.333374,0.427808,0.771245,-1.394042,-1.887790,0.217436,-0.747763,-1.368685
CTRL_037!!CCCTTAGCATATCTCT-1,0.142962,0.637160,0.248751,0.256609,0.590728,0.653836,-0.858147,1.163096,-0.457901,1.024236,...,0.119212,-0.947356,0.437092,-1.304053,1.748227,-1.613266,0.739681,-1.440679,-0.049319,0.656943
CTRL_037!!CTCCACACACTGAATC-1,0.134978,0.155870,0.432567,0.427428,0.369137,0.182925,0.369212,-0.561656,-1.264585,0.233709,...,0.164040,-0.686603,0.079379,-1.262676,2.887535,0.059416,-0.481227,0.070023,-0.701665,-0.624953
CTRL_037!!ACTATTCAGGCCCAAA-1,0.148326,1.805397,0.042957,0.040073,1.397144,1.749835,0.343753,-1.299991,0.926695,0.614490,...,-0.130636,-1.348338,0.326209,-2.553386,1.320922,-0.240211,-0.144704,0.654829,-0.302946,-1.092202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MSA_1436!!CAGCCAGAGTGGCGAT-1,0.167520,1.247546,0.108891,0.108538,0.964417,1.234339,1.120789,-0.497049,-0.801063,-0.675026,...,-0.601328,-0.108257,-0.000322,-1.142106,-1.013842,0.548040,-1.318534,-0.530553,1.245259,-0.451875
PD_7731!!ATACCTTCAACCCTCT-1,0.029703,-0.231167,0.581419,0.581116,0.235737,-0.204749,-0.614913,-0.054894,-0.606455,-0.070006,...,1.285756,0.753239,-0.382408,-1.571218,0.046974,0.156969,0.567595,-0.301017,-0.982560,0.595324
PD_7731!!AGGATCTAGCAATTAG-1,0.021157,-1.128515,0.875125,0.872494,0.059237,-1.138261,0.730471,-1.057453,0.234742,-1.610294,...,-0.205704,-0.006270,-1.200360,-0.173198,-1.130366,0.645251,1.129951,0.204125,0.962345,-0.287751


In [8]:
df_gene = scdrs.method.downstream_gene_analysis(adata, dict_df_score["PD"])

In [11]:
df_gene

,CORR,RANK
CELF2,0.534930,0
SLC8A1,0.499824,1
KCNMA1,0.496593,2
ARHGAP26,0.475355,3
CHST11,0.470886,4
...,...,...
RNF220,-0.514151,25729
MBP,-0.520511,25730
SLC44A1,-0.534805,25731
ST18,-0.546189,25732


In [10]:
# This was not used, so not provided
df_gene.to_csv("gwas/nalls/gene_correlation.csv")

# Downstream analyses, only microglia

In [20]:
adata = ad.read_h5ad("all_cells.h5ad")
anno = pd.read_table("annotation_micro.tsv", sep="\t")

In [17]:
adata.obs

,cell_type,disease
CTRL_037!!CATGAGTTCCACACAA-1,Astrocytes,CTRL
CTRL_039!!ACGGTTACAACACACT-1,Astrocytes,CTRL
CTRL_037!!CCCTTAGCATATCTCT-1,Astrocytes,CTRL
CTRL_037!!CTCCACACACTGAATC-1,Astrocytes,CTRL
CTRL_037!!ACTATTCAGGCCCAAA-1,Astrocytes,CTRL
...,...,...
MSA_1436!!CAGCCAGAGTGGCGAT-1,Inh. neurons,MSA
PD_7731!!ATACCTTCAACCCTCT-1,Inh. neurons,PD
PD_7731!!AGGATCTAGCAATTAG-1,Inh. neurons,PD
PD_7731!!AGGAAATTCTCGCGTT-1,Inh. neurons,PD


In [23]:
anno["anno"]

1       MIC_intermediate1
2       MIC_intermediate1
3        MIC_steady-state
4        MIC_steady-state
5        MIC_steady-state
              ...        
6925    MIC_intermediate2
6926    MIC_intermediate2
6927    MIC_intermediate2
6928    MIC_intermediate2
6929    MIC_intermediate1
Name: anno, Length: 6929, dtype: object

In [24]:
adata = adata[adata.obs.index.isin(anno["id"])]
adata.obs = adata.obs.reindex(anno["id"])
adata.obs["anno"] = pd.Categorical(anno["anno"])
adata.obs

,cell_type,disease,anno
id,,,
CTRL_037!!CCGGTAGTCCCGATCT-1,Microglia,CTRL,MIC_intermediate1
CTRL_039!!AGGCATTTCAACGCTA-1,Microglia,CTRL,MIC_intermediate1
CTRL_037!!TCTTAGTTCCCTAGGG-1,Microglia,CTRL,MIC_steady-state
CTRL_039!!TAACTTCTCGGTGTAT-1,Microglia,CTRL,MIC_steady-state
CTRL_037!!AACCCAAGTTCGTTCC-1,Microglia,CTRL,MIC_steady-state
...,...,...,...
PD_7044!!TGTAAGCTCAGTGATC-1,Microglia,PD,MIC_intermediate2
PD_7044!!AGAAATGAGCAGTACG-1,Microglia,PD,MIC_intermediate2
PD_7044!!AAGCGAGCATTCACCC-1,Microglia,PD,MIC_intermediate2


In [25]:
# Preprocessing
dict_gs = scdrs.util.load_gs(
    "gwas/nalls/munge.gs",
    src_species="human",
    dst_species="human",
    to_intersect=adata.var_names,
)

scdrs.preprocess(adata, n_mean_bin=20, n_var_bin=20, copy=False)

In [26]:
# Compute scores
dict_df_score = dict()
for trait in dict_gs:
    gene_list, gene_weights = dict_gs[trait]
    dict_df_score[trait] = scdrs.score_cell(
        data=adata,
        gene_list=gene_list,
        gene_weight=gene_weights,
        ctrl_match_key="mean_var",
        n_ctrl=1000,
        weight_opt="vs",
        return_ctrl_raw_score=False,
        return_ctrl_norm_score=True,
        verbose=True,
    )

# scdrs.method.score_cell summary:
    n_cell=6929, n_gene=25734,
    n_disease_gene=833,
    n_ctrl=1000, n_genebin=200,
    ctrl_match_key='mean_var',
    weight_opt='vs',
    return_ctrl_raw_score=False,
    return_ctrl_norm_score=True,
    random_seed=0, verbose=True,
    save_intermediate=None,
# scdrs.method.score_cell: use 833 overlapping genes for scoring


Computing control scores: 100%|██████████| 1000/1000 [01:04<00:00, 15.60it/s]


In [30]:
# Downstream analyses
df_stats = dict()
df_stats["cell_type"] = scdrs.method.downstream_group_analysis(
    adata=adata,
    df_full_score=dict_df_score["PD"],
    group_cols=["anno"],
)

df_stats["disease"] = scdrs.method.downstream_group_analysis(
    adata=adata,
    df_full_score=dict_df_score["PD"],
    group_cols=["disease"],
)

/work/miniconda3/envs/scdrs/lib/python3.12/site-packages/scdrs/method.py:957: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for group, df_group in df_meta.groupby(groupby):
/work/miniconda3/envs/scdrs/lib/python3.12/site-packages/scdrs/method.py:957: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for group, df_group in df_meta.groupby(groupby):


In [34]:
# This was not used, so not provided
dname = "disease"
with open("gwas/nalls/downstream_disease_microglia.tsv", 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=df_stats[dname].keys(), delimiter='\t')
    #writer.writeheader()
    writer.writerow(df_stats[dname])

In [28]:
df_stats["disease"]

{'disease':        n_cell  n_ctrl  assoc_mcp  assoc_mcz  hetero_mcp  hetero_mcz  \
 group                                                                 
 CTRL   3055.0  1000.0   0.379620   0.199115    0.608392   -0.402940   
 MSA     941.0  1000.0   0.501499  -0.022428    0.717283   -0.638360   
 PD     2933.0  1000.0   0.264735   0.544470    0.340659    0.247614   
 
        n_fdr_0.05  n_fdr_0.1  n_fdr_0.2  
 group                                    
 CTRL          0.0        1.0        2.0  
 MSA           0.0        0.0        0.0  
 PD            0.0        0.0        0.0  }

In [33]:
dname = "cell_type"
with open("gwas/nalls/downstream_celltype_microglia.tsv", 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=df_stats[dname].keys(), delimiter='\t')
    #writer.writeheader()
    writer.writerow(df_stats[dname])

In [32]:
df_stats["cell_type"]

{'anno':                    n_cell  n_ctrl  assoc_mcp  assoc_mcz  hetero_mcp  \
 group                                                                 
 MIC_activated       902.0  1000.0   0.933067  -1.343231    0.411588   
 MIC_intermediate1  1776.0  1000.0   0.963037  -1.658465    0.885115   
 MIC_intermediate2  1361.0  1000.0   0.032967   2.149652    0.834166   
 MIC_steady-state   2213.0  1000.0   0.076923   1.394042    0.838162   
 PVMs                677.0  1000.0   0.805195  -0.897032    0.498502   
 
                    hetero_mcz  n_fdr_0.05  n_fdr_0.1  n_fdr_0.2  
 group                                                            
 MIC_activated        0.170915         0.0        0.0        0.0  
 MIC_intermediate1   -1.155760         0.0        0.0        0.0  
 MIC_intermediate2   -0.912182         0.0        1.0        1.0  
 MIC_steady-state    -0.863341         0.0        0.0        1.0  
 PVMs                -0.022908         0.0        0.0        0.0  }

In [7]:
dict_df_score = dict()
dict_df_score["PD"] = pd.read_csv("gwas/nalls/PD.full_score.gz", sep = "\t")
dict_df_score["PD"] = dict_df_score["PD"].set_index(dict_df_score["PD"].iloc[:, 0])
dict_df_score["PD"].drop(dict_df_score["PD"].columns[0], axis = 1, inplace = True)
dict_df_score["PD"]

,raw_score,norm_score,mc_pval,pval,nlog10_pval,zscore,ctrl_norm_score_0,ctrl_norm_score_1,ctrl_norm_score_2,ctrl_norm_score_3,...,ctrl_norm_score_990,ctrl_norm_score_991,ctrl_norm_score_992,ctrl_norm_score_993,ctrl_norm_score_994,ctrl_norm_score_995,ctrl_norm_score_996,ctrl_norm_score_997,ctrl_norm_score_998,ctrl_norm_score_999
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
CTRL_037!!CATGAGTTCCACACAA-1,0.157448,1.406709,0.079920,0.083358,1.079052,1.382832,0.404423,-0.333098,0.357902,1.870497,...,-0.491384,1.319264,0.286928,-0.508785,-0.140749,-0.053322,0.384879,-0.049660,-1.808275,0.544533
CTRL_039!!ACGGTTACAACACACT-1,0.158600,2.936088,0.002997,0.002939,2.531734,2.754461,0.582191,-0.396215,-1.573622,1.361310,...,-1.571017,-1.622350,-0.333374,0.427808,0.771245,-1.394042,-1.887790,0.217436,-0.747763,-1.368685
CTRL_037!!CCCTTAGCATATCTCT-1,0.142962,0.637160,0.248751,0.256609,0.590728,0.653836,-0.858147,1.163096,-0.457901,1.024236,...,0.119212,-0.947356,0.437092,-1.304053,1.748227,-1.613266,0.739681,-1.440679,-0.049319,0.656943
CTRL_037!!CTCCACACACTGAATC-1,0.134978,0.155870,0.432567,0.427428,0.369137,0.182925,0.369212,-0.561656,-1.264585,0.233709,...,0.164040,-0.686603,0.079379,-1.262676,2.887535,0.059416,-0.481227,0.070023,-0.701665,-0.624953
CTRL_037!!ACTATTCAGGCCCAAA-1,0.148326,1.805397,0.042957,0.040073,1.397144,1.749835,0.343753,-1.299991,0.926695,0.614490,...,-0.130636,-1.348338,0.326209,-2.553386,1.320922,-0.240211,-0.144704,0.654829,-0.302946,-1.092202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MSA_1436!!CAGCCAGAGTGGCGAT-1,0.167520,1.247546,0.108891,0.108538,0.964417,1.234339,1.120789,-0.497049,-0.801063,-0.675026,...,-0.601328,-0.108257,-0.000322,-1.142106,-1.013842,0.548040,-1.318534,-0.530553,1.245259,-0.451875
PD_7731!!ATACCTTCAACCCTCT-1,0.029703,-0.231167,0.581419,0.581116,0.235737,-0.204749,-0.614913,-0.054894,-0.606455,-0.070006,...,1.285756,0.753239,-0.382408,-1.571218,0.046974,0.156969,0.567595,-0.301017,-0.982560,0.595324
PD_7731!!AGGATCTAGCAATTAG-1,0.021157,-1.128515,0.875125,0.872494,0.059237,-1.138261,0.730471,-1.057453,0.234742,-1.610294,...,-0.205704,-0.006270,-1.200360,-0.173198,-1.130366,0.645251,1.129951,0.204125,0.962345,-0.287751


In [36]:
adata

AnnData object with n_obs × n_vars = 6929 × 25734
    obs: 'cell_type', 'disease', 'anno'
    uns: 'neighbors', 'SCDRS_PARAM'
    obsm: 'X_pca'
    obsp: 'connectivities', 'distances'

In [38]:
df_gene = scdrs.method.downstream_gene_analysis(adata, dict_df_score["PD"])

In [40]:
df_gene[:50]

,CORR,RANK
P2RY12,0.178281,0
TMEM163,0.170895,1
MED12L,0.169363,2
MAP4K4,0.152851,3
KHDRBS3,0.150573,4
P2RY13,0.140340,5
BAG3,0.136211,6
SIPA1L2,0.129149,7
RAB29,0.124544,8
AKT3,0.120924,9


In [41]:
# This was not used, so not provided
df_gene.to_csv("gwas/nalls/gene_correlation_microglia.csv")